In this assignment, you will apply the ensemble ranking method to the the supplier selection scenario considered in assignment 4. The following code block import libraries you will need.

In [1]:
import pathlib

import numpy as np
import pandas as pd

import OM527_functions as omf

The following code block defines the `custom_grouper` function we used when considering spend analysis.

In [2]:
def custom_grouper(df, agg_dict, groupby_columns):
    '''
    This function groups the provided DataFrame, df, by the columns
    specified in the groupby_columns argument. The aggregations specified
    in the agg_dict dictionary are applied. Also, each numeric column in the 
    aggregated DataFrame is used to create a proportion column. The aggregated data
    is returned as a DataFrame sorted by the keys of the agg_dict
    dictionary, in the order they are specified, i.e., first key
    has a higher sort priority than the second, etc...
    '''
    
    grouped_df = df.groupby(groupby_columns).agg(agg_dict)
    
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
    
    numeric_columns = grouped_df.select_dtypes(include='number').columns.tolist()

    for column in numeric_columns:
        grouped_df[f'{column}_proportion'] = (grouped_df[column]/grouped_df[column].sum())
        
    grouped_df = grouped_df.sort_values(numeric_columns)

    return grouped_df

The following code block performs the data preparation steps described in assignment 3.

In [3]:
data_path = pathlib.Path('data', 'rw_supplier_data.csv')
rw_data = pd.read_csv(data_path)

relevant_naics = [493120]
rw_data = rw_data[rw_data['Primary NAICS'].isin(relevant_naics)]

rw_data['Location Sales Volume Actual'] = rw_data['Location Sales Volume Actual'].str.replace('$', '')
rw_data['Location Sales Volume Actual'] = rw_data['Location Sales Volume Actual'].str.replace(',', '')
rw_data['Location Sales Volume Actual'] = pd.to_numeric(rw_data['Location Sales Volume Actual'])



# Define and normalize weights
weights = {'Credit Score Num': 9, 
           'Location Score': 7,
           'Location Sales Volume Actual': 4}

credit_score_mapper = {
    'A+': 1,
    'A': 0.8,
    'B+': 0.6,
    'B': 0.4,
    'C+': 0.2,
}
rw_data['Credit Score Num'] = rw_data['Credit Score Alpha'].map(credit_score_mapper)

unique_states = rw_data['State'].unique().tolist()
state_mapper = {}
for state in unique_states:
    if (state == 'AL'):
        state_mapper[state] = 1
    else:
        state_mapper[state] = 0.5
rw_data['Location Score'] = rw_data['State'].map(state_mapper)

groupby_columns = ['Company Name']
agg_dict = {
    'Location Sales Volume Actual': ['sum'],
    'Credit Score Num': ['mean'],
    'Location Score': ['mean'],
}
rw_data = rw_data.groupby(groupby_columns).agg(agg_dict)
rw_data.columns = agg_dict.keys()
rw_data = rw_data.reset_index()

sales_volume_array = rw_data['Location Sales Volume Actual'].values
rw_data['Location Sales Volume Actual'] = omf.mcdm.normalize_array(sales_volume_array)

weights = {'Credit Score Num': 9, 
           'Location Score': 7,
           'Location Sales Volume Actual': 4}

weights = omf.mcdm.normalize_weights_dictionary(weights)

Use the `robust_ranking` function available in the `mcdm_functions` module to identify suppliers that occur in the top 5 by each ranking method, e.g., weighted sum and weighted product, when considering 100 perturbations of the weights that are within 20% of the weights specified in the `weights` object. **(100 points)**